The aim of this notebook is to make use of the word2vec model to find similar songs

In [1]:
import pandas as pd
import numpy as np
import gensim.models.word2vec as w2v
import multiprocessing
import os
import re
import pprint
import sklearn.manifold
import matplotlib.pyplot as plt

Though non english artists were removed, the dataset contained Hindi lyrics of Lata Mangeshkar written in English. Therefore, I decided to remove all songs sung by her.

In [2]:
songs = pd.read_csv("C:/Users/Marino/Documents/UNIVERSIDAD/MasterBD/AnalisisDatosNoEstructurados/Practica/4_TEXT/data/hhgroups_merge_28_05.csv", header=0)
songs.head()

,id,artista,cancion,album,letra,anyo,visitas
0,0,Denom,Machete (con Jarfaiter y Gente jodida),Medicina,"Para su nuevo disco ""Medicina"", Denom ha vuelt...",2019,126
1,1,Denom,Vacío (con Ivo Incuerdo),Medicina,"[Denom]\nYo que quería, yo que pedía vida,\nSe...",2019,361
2,2,Denom,El orgullo es fiel (con Juancho Marqués y Elio...,Medicina,"""El orgullo es fiel"" es uno de los cortes incl...",2019,262
3,3,Denom,Mueve mueve (con Fernandocosta),Medicina,"[Estribillo: Denom] (x2)\nMueve, mueve, mueve,...",2019,578
4,4,Jaro Desperdizio,Insomnia,"Sin álbum, es un vídeo suelto","[Estribillo]\nY en esta noche, ¿Quién me arrop...",2019,219


In [3]:
artistas = songs["artista"].unique()

In [4]:
artistas.size

1609

To train the word2vec model, we first need to build its vocabulary. To do that, I iterated over each song and added it to an array that can later be fed to the model.

En un pricipio se pensó en hacer un replace de los caracteres especiales a la hora de ejecutar el corpus de la siguiente manera:


In [5]:
letras = []
for song in songs['letra']:
    words = re.sub("[^a-zA-Z]", ' ', str(song))
    words = words.lower().split()
    letras.append(words)
letras[0:1]  

[['para',
  'su',
  'nuevo',
  'disco',
  'medicina',
  'denom',
  'ha',
  'vuelto',
  'a',
  'contar',
  'con',
  'la',
  'colaboraci',
  'n',
  'vocal',
  'de',
  'jarfaiter',
  'con',
  'qui',
  'n',
  'ya',
  'ha',
  'colaborado',
  'anteriormente',
  'en',
  'el',
  'tema',
  'machete',
  'corte',
  'en',
  'el',
  'que',
  'tambien',
  'participa',
  'el',
  'grupo',
  'gente',
  'jodida',
  'de',
  'la',
  'instrumental',
  'se',
  'encarga',
  'narcosbeats',
  'tienes',
  'ya',
  'la',
  'letra',
  'para',
  'este',
  'tema',
  'ay',
  'danos',
  'y',
  'env',
  'anosla']]

Como se puede comprobar, las tildes las considera como caracteres especiales, por lo que habría que cambiar la expresión del regex a:  "[^\w+]"

Al hacer el songvector, da un error al no reconocer palabras referenciadas entre comillas,como "Medicina" que es el nombre de un disco, por lo que se decidió cambiar el formato de la columna "letra" en el dataframe de origen:

In [6]:
letras = songs["letra"].replace("[^\w+]", " ", regex = True) 
letras

0       Para su nuevo disco  Medicina   Denom ha vuelt...
1        Denom  Yo que quería  yo que pedía vida  Se p...
2        El orgullo es fiel  es uno de los cortes incl...
3        Estribillo  Denom   x2  Mueve  mueve  mueve  ...
4        Estribillo  Y en esta noche   Quién me arropa...
                              ...                        
9320    Cuestiones de fianzas  tras de más desconfianz...
9321    Quiero decirte tantas cosas que no pude  asume...
9322     Porta  Tenemos que hablar  esto tiene que cam...
9323     Estribillo   En inglés    Nunca llegarás a en...
9324    Soy yo  hello  soy el Arbel López  negrata est...
Name: letra, Length: 9325, dtype: object

In [7]:
songs["letra"] = songs["letra"].replace("[^\w+]", " ", regex = True) 
songs.head()

,id,artista,cancion,album,letra,anyo,visitas
0,0,Denom,Machete (con Jarfaiter y Gente jodida),Medicina,Para su nuevo disco Medicina Denom ha vuelt...,2019,126
1,1,Denom,Vacío (con Ivo Incuerdo),Medicina,Denom Yo que quería yo que pedía vida Se p...,2019,361
2,2,Denom,El orgullo es fiel (con Juancho Marqués y Elio...,Medicina,El orgullo es fiel es uno de los cortes incl...,2019,262
3,3,Denom,Mueve mueve (con Fernandocosta),Medicina,Estribillo Denom x2 Mueve mueve mueve ...,2019,578
4,4,Jaro Desperdizio,Insomnia,"Sin álbum, es un vídeo suelto",Estribillo Y en esta noche Quién me arropa...,2019,219


In [8]:
text_corpus = []
for song in songs['letra']:
    #words = re.sub("[^\w+]", ' ', str(song))
    words = song.lower().split()
    text_corpus.append(words)


# Dimensionality of the resulting word vectors.
#more dimensions, more computationally expensive to train
#but also more accurate
#more dimensions = more generalized
num_features = 50
# Minimum word count threshold.
min_word_count = 1

# Number of threads to run in parallel.
#more workers, faster we train
num_workers = multiprocessing.cpu_count()

# Context window length.
context_size = 7


downsampling = 1e-1

# Seed for the RNG, to make the results reproducible.
#random number generator
#deterministic, good for debugging
seed = 1

songs2vec = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

songs2vec.build_vocab(text_corpus)
print (len(text_corpus))

9325


In [9]:
text_corpus[0]

['para',
 'su',
 'nuevo',
 'disco',
 'medicina',
 'denom',
 'ha',
 'vuelto',
 'a',
 'contar',
 'con',
 'la',
 'colaboración',
 'vocal',
 'de',
 'jarfaiter',
 'con',
 'quién',
 'ya',
 'ha',
 'colaborado',
 'anteriormente',
 'en',
 'el',
 'tema',
 'machete',
 'corte',
 'en',
 'el',
 'que',
 'tambien',
 'participa',
 'el',
 'grupo',
 'gente',
 'jodida',
 'de',
 'la',
 'instrumental',
 'se',
 'encarga',
 'narcosbeats',
 'tienes',
 'ya',
 'la',
 'letra',
 'para',
 'este',
 'tema',
 'ayúdanos',
 'y',
 'envíanosla']

In [10]:
import time
start_time = time.time()



songs2vec.train(text_corpus, total_examples=songs2vec.corpus_count, epochs=2)

if not os.path.exists("trained"):
    os.makedirs("trained")

songs2vec.save(os.path.join("trained", "songs2vectors.w2v"))

print("--- %s seconds ---" % (time.time() - start_time))

--- 46.48275327682495 seconds ---


In [11]:
songs2vec = w2v.Word2Vec.load(os.path.join("trained", "songs2vectors.w2v"))

#### Let's explore our model

Find similar words

In [12]:
songs2vec.wv.most_similar("amor")

[('odio', 0.9110756516456604),
 ('rencor', 0.9035030603408813),
 ('platónico', 0.8948013782501221),
 ('dolor', 0.8946930170059204),
 ('desamor', 0.8880473971366882),
 ('afecto', 0.8863727450370789),
 ('egoísmo', 0.8817657828330994),
 ('sentimiento', 0.8777220845222473),
 ('cariño', 0.8768129348754883),
 ('silencio', 0.8734211325645447)]

In [13]:
songs2vec.wv.most_similar("persona")

[('pieza', 0.8978850841522217),
 ('diosa', 0.8902904987335205),
 ('promesa', 0.8823827505111694),
 ('madura', 0.8821364641189575),
 ('linda', 0.8808770775794983),
 ('bestia', 0.8801291584968567),
 ('etapa', 0.8789024353027344),
 ('ironía', 0.8777978420257568),
 ('pequeña', 0.8776611685752869),
 ('mujer', 0.8767144680023193)]

Words out of context

In [14]:
songs2vec.wv.doesnt_match("feliz amor mesa odio".split())

C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\gensim\models\keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'mesa'

In [15]:
songs2vec.most_similar(positive=['mujer', 'rey'], negative=['hombre'])
#reina

C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('zorra', 0.8323769569396973),
 ('rata', 0.8115664124488831),
 ('guarra', 0.8113574385643005),
 ('cerda', 0.8070788979530334),
 ('cabra', 0.8064299821853638),
 ('estrella', 0.8058162927627563),
 ('rubia', 0.8032612800598145),
 ('cruz', 0.8003140091896057),
 ('gurú', 0.7990587949752808),
 ('sorpresa', 0.7972445487976074)]

En este caso, debido a que el corpus utilizado contiene canciones de estilos como el reggaeton o el rap, las palabras que salen similares a "reina", no son las esperadas

Semantic distance between words

In [16]:
def nearest_similarity_cosmul(start1, end1, end2):
    similarities = songs2vec.wv.most_similar_cosmul(
        positive=[end2, start1],
        negative=[end1]
    )
    start2 = similarities[0][0]
    print("{0} es a {1}, lo que {2} es a {3}".format(start1, end1, start2, end2))

In [17]:
nearest_similarity_cosmul("paris", "francia", "españa")
nearest_similarity_cosmul("paris", "francia", "madrid")

paris es a francia, lo que sevilla es a españa
paris es a francia, lo que barcelona es a madrid


With the word vector embeddings in place, it is now time to calculate the normalised vector sum of each song. This process can take some time since it has to be done for each of 57,000 songs.

In [18]:
def songVector(row):
    vector_sum = 0
    words = re.sub("[^\w+]", ' ', str(row))
    words = row.lower().split()
    for word in words:
        vector_sum = vector_sum + songs2vec[word]
    vector_sum = vector_sum.reshape(1,-1)
    normalised_vector_sum = sklearn.preprocessing.normalize(vector_sum)
    return normalised_vector_sum

import sys
import time
start_time = time.time()

songs['song_vector'] = songs['letra'].apply(songVector)




C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method 

C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method 

C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method 

C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method 

C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method 

...

C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method 

C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method 

C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method 

C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method 

C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Archivos de programa\Anaconda\envs\tfenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method 

**t-sne and random song selection** 

The songs have 50 dimensions each. Application of t-sne is memory intensive and hence it is slightly easier on the computer to use a random sample of the 57,000 songs.

In [19]:
song_vectors = []
from sklearn.model_selection import train_test_split

train, test = train_test_split(songs, test_size = 0.9)


for song_vector in train['song_vector']:
    song_vectors.append(song_vector)

train.head(10)

,id,artista,cancion,album,letra,anyo,visitas,song_vector
3425,15,Sr. Zambrana,No es dificil soñar,Sin álbum,Joder que sueño coño Niño levantate ya y apaga...,0,1786,"[[-0.11412063, -0.17145993, 0.015032126, -0.06..."
8864,34,Frescolate,No fue amor (con Genesis),R.A.P.,Querida te llego el día o mejor dicho la noche...,2012,1840,"[[-0.10549686, -0.14880225, 0.0084618395, -0.0..."
4616,21,Recycled J y Selecta,Aunque digan que yo,City pop,Estribillo Aunque digan que yo Soy la oveja...,2019,6460,"[[-0.11655058, -0.18158832, 0.03295755, -0.094..."
2222,22,Flowklorikos,Sucio (con Carlos Talavera),Donde duele inspira,Carlos Talavera Son historias pa contar otr...,2009,3994,"[[-0.1016627, -0.15024552, 0.020191938, -0.096..."
6405,50,Juaninacka,Básico 3,"Sin álbum, es un vídeo suelto",Dime qué ves cuando te miras Buscando el se...,2018,1661,"[[-0.13529265, -0.1380708, -0.0076722326, -0.0..."
9141,36,Pablo Hasél,Muerte a los borbones,El Che disparaba,Vuelvo a desmontar ese cuento para niños de qu...,2011,1828,"[[-0.13459815, -0.17257406, 0.004944725, -0.06..."
333,3,Elemento Clab,"Primera pagina (con Edu Segura, Niko mendez y ...",Diarios de un nomada,Vengo escribiendo yo desde hace tiempo todo l...,2011,925,"[[-0.10513381, -0.16164204, 0.00981952, -0.078..."
7719,44,Neim,Están lloviendo dudas (con Arkano),Orígenes acuáticos,Where are you Where are you honey Where are...,2014,487,"[[-0.10027804, -0.14624809, 0.0660222, -0.0993..."
8709,44,Freshmakers,Demasiado corazón,No gravity,Estribillo No nos conoces welcome to the ju...,2013,1183,"[[-0.10552605, -0.14545508, 0.02667831, -0.064..."
3983,23,CPV,La fundación de la rima,"Madrid, zona bruta",Meswy Sed bienvenidos a mi organización Bien...,1994,2266,"[[-0.11752852, -0.1855993, 0.013596675, -0.075..."


In [20]:
songs.size

74600

In [21]:
len(song_vectors)

932

I had a fairly measly 4gb machine and wasn't able to generate a more accurate model. However, one can play around with the number of iterations, learning rate and other factors to fit the model better. If you have too many dimensions (~300+), it might make sense to use PCA first and then t-sne.

In [22]:
X = np.array(song_vectors).reshape((932, 50))

start_time = time.time()
tsne = sklearn.manifold.TSNE(n_components=2, n_iter=250, random_state=0, verbose=2)

all_word_vectors_matrix_2d = tsne.fit_transform(X)

print("--- %s seconds ---" % (time.time() - start_time))

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 932 samples in 0.008s...
[t-SNE] Computed neighbors for 932 samples in 0.084s...
[t-SNE] Computed conditional probabilities for sample 932 / 932
[t-SNE] Mean sigma: 0.029774
[t-SNE] Computed conditional probabilities in 0.053s
[t-SNE] Iteration 50: error = 74.4799042, gradient norm = 0.3190045 (50 iterations in 0.355s)
[t-SNE] Iteration 100: error = 75.2784882, gradient norm = 0.3360281 (50 iterations in 0.359s)
[t-SNE] Iteration 150: error = 76.1286392, gradient norm = 0.3111066 (50 iterations in 0.369s)
[t-SNE] Iteration 200: error = 76.8295898, gradient norm = 0.3106605 (50 iterations in 0.401s)
[t-SNE] Iteration 250: error = 77.0928879, gradient norm = 0.2944887 (50 iterations in 0.479s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 77.092888
[t-SNE] KL divergence after 251 iterations: 17976931348623157081452742373170435679807056752584499659891747680315726078002853876058955863276687817154045895351438246

In [23]:
df=pd.DataFrame(all_word_vectors_matrix_2d,columns=['X','Y'])

df.head(10)

train.head()

df.reset_index(drop=True, inplace=True)
train.reset_index(drop=True, inplace=True)

Joining two dataframes to obtain each song's corresponding X,Y co-ordinate.

In [24]:
two_dimensional_songs = pd.concat([train, df], axis=1)

two_dimensional_songs.head()

,id,artista,cancion,album,letra,anyo,visitas,song_vector,X,Y
0,15,Sr. Zambrana,No es dificil soñar,Sin álbum,Joder que sueño coño Niño levantate ya y apaga...,0,1786,"[[-0.11412063, -0.17145993, 0.015032126, -0.06...",0.078709,0.190395
1,34,Frescolate,No fue amor (con Genesis),R.A.P.,Querida te llego el día o mejor dicho la noche...,2012,1840,"[[-0.10549686, -0.14880225, 0.0084618395, -0.0...",0.301070,0.097261
2,21,Recycled J y Selecta,Aunque digan que yo,City pop,Estribillo Aunque digan que yo Soy la oveja...,2019,6460,"[[-0.11655058, -0.18158832, 0.03295755, -0.094...",0.095304,-0.051830
3,22,Flowklorikos,Sucio (con Carlos Talavera),Donde duele inspira,Carlos Talavera Son historias pa contar otr...,2009,3994,"[[-0.1016627, -0.15024552, 0.020191938, -0.096...",0.246444,0.060697
4,50,Juaninacka,Básico 3,"Sin álbum, es un vídeo suelto",Dime qué ves cuando te miras Buscando el se...,2018,1661,"[[-0.13529265, -0.1380708, -0.0076722326, -0.0...",0.355492,0.016113


In [25]:
x = two_dimensional_songs.X
x

0      0.078709
1      0.301070
2      0.095304
3      0.246444
4      0.355492
         ...   
927    0.162844
928    0.119710
929    0.402521
930   -0.066516
931    0.295439
Name: X, Length: 932, dtype: float32

**Plotting the results**

Using plotly, I plotted the results so that it becomes easier to explore similar songs based on their colors and clusters.

In [26]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

import plotly.graph_objs as go

trace1 = go.Scatter(
    y = two_dimensional_songs['Y'],
    x = two_dimensional_songs['X'],
    text = two_dimensional_songs['cancion'],
    mode='markers',
    marker=dict(
        size= 8,#'7',
        color = two_dimensional_songs['anyo'], #set color equal to a variable
        colorscale='Viridis',
        showscale=True
    )
)
data = [trace1]

iplot(data)
#Grafico1

In [27]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure(data=go.Scatter(
    y = two_dimensional_songs['Y'],
    x = two_dimensional_songs['X'],
    text = two_dimensional_songs['artista']+ "_"+two_dimensional_songs['cancion'] ,
    mode='markers',
    marker=dict(
        size= 8,#'7',
        color = two_dimensional_songs['visitas'], #set color equal to a variable
        colorscale='Viridis',
        showscale=True
    )
))

fig.show()
#Grafico2

### LOOK FOR COMMON SONGS AND ANALYZE THE TEXT

In [28]:
import plotly.express as px

fig = px.scatter_3d(two_dimensional_songs,x='X', y='Y', z='visitas',
              color='anyo')
fig.show()
#Grafico3

Salen canciones con el valor "anyo" igual a 0, lo que desvirtúa la escala por lo que introducimos los años de esas canciones a mano:

In [29]:
two_dimensional_songs.anyo.unique()

array([   0, 2012, 2019, 2009, 2018, 2011, 2014, 2013, 1994, 2017, 2015,
       2010, 2006, 2005, 2008, 2003, 2007, 2000, 2016, 2002, 2004, 2001,
       1999, 1998, 1997], dtype=int64)

In [30]:
two_dimensional_songs.loc[two_dimensional_songs["anyo"]==0]

,id,artista,cancion,album,letra,anyo,visitas,song_vector,X,Y
0,15,Sr. Zambrana,No es dificil soñar,Sin álbum,Joder que sueño coño Niño levantate ya y apaga...,0,1786,"[[-0.11412063, -0.17145993, 0.015032126, -0.06...",0.078709,0.190395
41,49,Hermanos Herm&eacute;ticos,Nadie Que Me Quiera,Sin álbum,Aaron Baliti Amargas juergas pongo el rad...,0,2922,"[[-0.1172339, -0.17878383, 0.025312386, -0.081...",0.044117,-0.069219
92,1,Legendario,Apostar,Sin álbum,Estribillo Todo tu dinero es agua la vida es...,0,1571,"[[-0.12663269, -0.17828594, 0.009086142, -0.07...",-0.119447,-0.044247
100,17,Arte MC,Deseos (con Darco),Sin álbum,Arte Mc Yeah es Darco con mas Arte Mc 2 0 0 ...,0,1210,"[[-0.12255546, -0.15425566, 0.011857719, -0.08...",11.835432,-0.118461
142,42,El Chojin,Invasion,Sin álbum,Míranos caer peleas a emprender yo tengo el ...,0,1461,"[[-0.11660494, -0.16439249, 0.006060947, -0.06...",-0.223127,0.424408
151,10,Zenit,Competi (con Artes 1/29),Sin álbum,Arturo majestad podria ser uno de los caballer...,0,2529,"[[-0.1157962, -0.1686807, 0.0067519723, -0.077...",-0.177767,-0.052091
156,20,Eisman y Rhoe,Donde mas duele [Base Internet],"Sin álbum, es un tema suelto",Estribillo x2 Seguimos sin palabras las le...,0,1471,"[[-0.11952805, -0.15072526, 0.019624002, -0.08...",-1.635381,-13.526676
160,4,Legendario,Elias Maluko (con Shotta),Sin álbum,Shotta Yeee Shotta Legendario Legendario L...,0,2449,"[[-0.1067965, -0.1922878, 0.012604991, -0.0605...",-0.268491,0.022892
161,30,BeMoL,Yimi,Sin álbum,Es la historia de Yimi akel chaval de pelo kl...,0,1223,"[[-0.103241, -0.17250624, 0.03681701, -0.07510...",0.117261,-0.023436
169,2,El Chojin,Algo,Sin álbum,Fue como andar descalzo sobre clavos fue como...,0,3535,"[[-0.11528875, -0.16016302, 0.022874521, -0.10...",0.173487,0.068303


In [ ]:
two_dimensional_songs.at[19,'anyo']=2004
two_dimensional_songs.at[41,'anyo']=2008
two_dimensional_songs.at[46,'anyo']=2003
two_dimensional_songs.at[63,'anyo']=2004
two_dimensional_songs.at[64,'anyo']=1999
two_dimensional_songs.at[65,'anyo']=1999
two_dimensional_songs.at[68,'anyo']=2003
two_dimensional_songs.at[69,'anyo']=2005
two_dimensional_songs.at[94,'anyo']=2001
two_dimensional_songs.at[124,'anyo']=2006
two_dimensional_songs.at[126,'anyo']=2008
two_dimensional_songs.at[150,'anyo']=2003
two_dimensional_songs.at[163,'anyo']=2003
two_dimensional_songs.at[166,'anyo']=2004
two_dimensional_songs.at[214,'anyo']=2003
two_dimensional_songs.at[219,'anyo']=2001
two_dimensional_songs.at[224,'anyo']=2001
two_dimensional_songs.at[257,'anyo']=2005
two_dimensional_songs.at[262,'anyo']=2002
two_dimensional_songs.at[264,'anyo']=2002
two_dimensional_songs.at[281,'anyo']=2005
two_dimensional_songs.at[333,'anyo']=2007
two_dimensional_songs.at[345,'anyo']=2004
two_dimensional_songs.at[370,'anyo']=2005
two_dimensional_songs.at[455,'anyo']=2005
two_dimensional_songs.at[459,'anyo']=2004
two_dimensional_songs.at[491,'anyo']=2005
two_dimensional_songs.at[508,'anyo']=2002
two_dimensional_songs.at[522,'anyo']=2005
two_dimensional_songs.at[525,'anyo']=2001
two_dimensional_songs.at[543,'anyo']=2005
two_dimensional_songs.at[561,'anyo']=2008
two_dimensional_songs.at[569,'anyo']=1999
two_dimensional_songs.at[598,'anyo']=2002
two_dimensional_songs.at[704,'anyo']=2004
two_dimensional_songs.at[711,'anyo']=2004
two_dimensional_songs.at[738,'anyo']=2005
two_dimensional_songs.at[740,'anyo']=2005
two_dimensional_songs.at[742,'anyo']=2005
two_dimensional_songs.at[791,'anyo']=2005
two_dimensional_songs.at[796,'anyo']=2005
two_dimensional_songs.at[823,'anyo']=2005
two_dimensional_songs.at[842,'anyo']=2005
two_dimensional_songs.at[879,'anyo']=2010

Al ser el proceso aleatorio, y ejecutarlo de nuevo, se cambia las canciones mostradas por lo que se decide excluir aquellas canciones cuyo valor de "anyo" sea 0

In [34]:
two_dimensional_songs = two_dimensional_songs.loc[two_dimensional_songs["anyo"]!=0]
two_dimensional_songs

,id,artista,cancion,album,letra,anyo,visitas,song_vector,X,Y
1,34,Frescolate,No fue amor (con Genesis),R.A.P.,Querida te llego el día o mejor dicho la noche...,2012,1840,"[[-0.10549686, -0.14880225, 0.0084618395, -0.0...",0.301070,0.097261
2,21,Recycled J y Selecta,Aunque digan que yo,City pop,Estribillo Aunque digan que yo Soy la oveja...,2019,6460,"[[-0.11655058, -0.18158832, 0.03295755, -0.094...",0.095304,-0.051830
3,22,Flowklorikos,Sucio (con Carlos Talavera),Donde duele inspira,Carlos Talavera Son historias pa contar otr...,2009,3994,"[[-0.1016627, -0.15024552, 0.020191938, -0.096...",0.246444,0.060697
4,50,Juaninacka,Básico 3,"Sin álbum, es un vídeo suelto",Dime qué ves cuando te miras Buscando el se...,2018,1661,"[[-0.13529265, -0.1380708, -0.0076722326, -0.0...",0.355492,0.016113
5,36,Pablo Hasél,Muerte a los borbones,El Che disparaba,Vuelvo a desmontar ese cuento para niños de qu...,2011,1828,"[[-0.13459815, -0.17257406, 0.004944725, -0.06...",-0.097211,0.030870
...,...,...,...,...,...,...,...,...,...,...
927,33,Shinoflow,La vedadera verdad,Adelantando relojes,Estribillo Yo soy el fruto de los árboles La...,2008,2611,"[[-0.12570329, -0.15721908, 0.011075871, -0.07...",0.162844,-0.025345
928,9,Crema,Juntar una vida,Agorazein,Llamadlo opera prima traigo mi alma en versión...,2008,15741,"[[-0.11218148, -0.17206286, 0.01435669, -0.076...",0.119710,-0.039982
929,41,Pekado,Cierro los ojos,Pulso,Estribillo Cierro los ojos y veo su cara Es...,2015,509,"[[-0.13792433, -0.16411698, 0.027551662, -0.08...",0.402521,0.109048
930,13,Larah Fémina y Sator,Pasion,Personalidad,Larah Es algo más que rap que hip hop Es mú...,2009,1772,"[[-0.103655815, -0.171152, 0.0019679908, -0.06...",-0.066516,0.102897


In [35]:
two_dimensional_songs.anyo.unique()

array([2012, 2019, 2009, 2018, 2011, 2014, 2013, 1994, 2017, 2015, 2010,
       2006, 2005, 2008, 2003, 2007, 2000, 2016, 2002, 2004, 2001, 1999,
       1998, 1997], dtype=int64)

In [36]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

import plotly.graph_objs as go

trace1 = go.Scatter(
    y = two_dimensional_songs['Y'],
    x = two_dimensional_songs['X'],
    text = two_dimensional_songs['cancion'],
    mode='markers',
    marker=dict(
        size= 8,#'7',
        color = two_dimensional_songs['anyo'], #set color equal to a variable
        colorscale='Viridis',
        showscale=True
    )
)
data = [trace1]

iplot(data)
#Grafico4

In [37]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure(data=go.Scatter(
    y = two_dimensional_songs['Y'],
    x = two_dimensional_songs['X'],
    text = two_dimensional_songs['artista']+ "_"+two_dimensional_songs['cancion'] ,
    mode='markers',
    marker=dict(
        size= 8,#'7',
        color = two_dimensional_songs['visitas'], #set color equal to a variable
        colorscale='Viridis',
        showscale=True
    )
))

fig.show()
#Grafico5

In [38]:
import plotly.express as px

fig = px.scatter_3d(two_dimensional_songs,x='X', y='Y', z='visitas',
              color='anyo')
fig.show()
#Grafico6

Se puede observar que no existe una correlación directa entre visitas y años, aunque sí se puede ver cómo las canciones más antiguas, generalmente tienen menos visitas porque están más abajo en el eje x 

A continuación vamos a crear una variable categórica que contemple si la canción formaba parte de un álbum o si por el contrario era un single. 

In [43]:
two_dimensional_songs.album.unique()

array(['R.A.P.', 'City pop', 'Donde duele inspira',
       'Sin álbum, es un vídeo suelto', 'El Che disparaba',
       'Diarios de un nomada', 'Orígenes acuáticos', 'No gravity',
       'Madrid, zona bruta', 'Bajo esta máscara', 'Alguien normal',
       'Misantropía', 'Random', 'Versatili-Rap',
       'Vivir para contarlo/Haciendo lo nuestro', '2005',
       'Coordenadas 4T+', 'Alquimia', 'Working class', 'Proyecto renacer',
       'Sentimiento mediocre', 'Kamikaze', 'LB Finest', 'Alevosía',
       'Silencio a gritos', 'Cierre', 'Coincidencia', 'A puerto',
       'Cosas que pasan, que no pasan y que deberían pasar', 'Maleza',
       'Autocheck', 'Abracadaver', 'Dejemonos de tonterias',
       'Extremely flammable', 'Contradicziones',
       'La estanquera de Saigón', 'La última nota', 'Caso omiso',
       'Hombre de oro', 'Pandemia', 'Ataraxia (A plena luz del día)',
       'Caleidoscopio', 'Mirlo blanco', "This unruly mess I've made",
       'Striptease', 'Debut 2.0', '10', 'Raíces', 

In [54]:
tabla = two_dimensional_songs[two_dimensional_songs['album'].str.contains("Sin álbum")]

In [55]:
tabla.album.unique()

array(['Sin álbum, es un vídeo suelto', 'Sin álbum, es un tema suelto',
       'Sin álbum'], dtype=object)

In [71]:
import numpy as np
two_dimensional_songs['album_cat'] = np.where(two_dimensional_songs['album'].str.contains("Sin álbum"),"0", "1")
two_dimensional_songs

,id,artista,cancion,album,letra,anyo,visitas,song_vector,X,Y,album_cat
1,34,Frescolate,No fue amor (con Genesis),R.A.P.,Querida te llego el día o mejor dicho la noche...,2012,1840,"[[-0.10549686, -0.14880225, 0.0084618395, -0.0...",0.301070,0.097261,1
2,21,Recycled J y Selecta,Aunque digan que yo,City pop,Estribillo Aunque digan que yo Soy la oveja...,2019,6460,"[[-0.11655058, -0.18158832, 0.03295755, -0.094...",0.095304,-0.051830,1
3,22,Flowklorikos,Sucio (con Carlos Talavera),Donde duele inspira,Carlos Talavera Son historias pa contar otr...,2009,3994,"[[-0.1016627, -0.15024552, 0.020191938, -0.096...",0.246444,0.060697,1
4,50,Juaninacka,Básico 3,"Sin álbum, es un vídeo suelto",Dime qué ves cuando te miras Buscando el se...,2018,1661,"[[-0.13529265, -0.1380708, -0.0076722326, -0.0...",0.355492,0.016113,0
5,36,Pablo Hasél,Muerte a los borbones,El Che disparaba,Vuelvo a desmontar ese cuento para niños de qu...,2011,1828,"[[-0.13459815, -0.17257406, 0.004944725, -0.06...",-0.097211,0.030870,1
...,...,...,...,...,...,...,...,...,...,...,...
927,33,Shinoflow,La vedadera verdad,Adelantando relojes,Estribillo Yo soy el fruto de los árboles La...,2008,2611,"[[-0.12570329, -0.15721908, 0.011075871, -0.07...",0.162844,-0.025345,1
928,9,Crema,Juntar una vida,Agorazein,Llamadlo opera prima traigo mi alma en versión...,2008,15741,"[[-0.11218148, -0.17206286, 0.01435669, -0.076...",0.119710,-0.039982,1
929,41,Pekado,Cierro los ojos,Pulso,Estribillo Cierro los ojos y veo su cara Es...,2015,509,"[[-0.13792433, -0.16411698, 0.027551662, -0.08...",0.402521,0.109048,1
930,13,Larah Fémina y Sator,Pasion,Personalidad,Larah Es algo más que rap que hip hop Es mú...,2009,1772,"[[-0.103655815, -0.171152, 0.0019679908, -0.06...",-0.066516,0.102897,1


In [73]:
two_dimensional_songs['album_cat'].unique()

array(['1', '0'], dtype=object)

In [78]:
import plotly.express as px

fig = px.scatter_3d(two_dimensional_songs,x='X', y='Y', z='visitas',
              color='album_cat', size='anyo')
fig.show()
#Grafico7

Se puede observar como lanzar un single, generalmente es negativo en cuanto a visitas ya que aparecen bastante abajo en el plot.